# 📊 二進位購物資料探索任務

In [ ]:
# 匯入套件
import os
import sys
import platform
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns

# 🈶 中文字型設定（支援 Colab 與本機）
font_path = "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc"
if os.path.exists(font_path):
    font_prop = fm.FontProperties(fname=font_path)
    plt.rcParams['font.family'] = font_prop.get_name()
else:
    print("⚠️ 字型未找到，使用預設設定")

# 修正負號顯示為亂碼
plt.rcParams['axes.unicode_minus'] = False

# 📁 Colab：自動 clone 並修正目錄層級
if "google.colab" in sys.modules:
    repo = "binary-shopping-explorer"
    if not os.path.exists(repo):
        !git clone https://github.com/nouclass111270/binary-shopping-explorer.git
    # 避免多層 clone 導致資料路徑錯誤
    while not os.path.exists(os.path.join("data", "shopbi_explore3k.csv")):
        os.chdir(repo)

# 📄 資料路徑與讀取檢查
file_path = os.path.join("data", "shopbi_explore3k.csv")
print("目前讀取資料的路徑為：", os.path.abspath(file_path))

if not os.path.exists(file_path):
    raise FileNotFoundError(f"❌ 找不到檔案：{file_path}\n請確認 data 資料夾與檔案名稱是否正確")

# 載入資料
# 資料來源為模擬的 3,000+ 筆購物記錄，包含顧客、商品、分類與購物步驟
df = pd.read_csv(file_path, encoding='utf-8-sig')

# 建立商品接續欄位 next_product（依 session_id 與 sequence_order 排序）
df = df.sort_values(by=['session_id', 'sequence_order'])
df['next_product'] = df.groupby('session_id')['product'].shift(-1)

<details open>
<summary><strong>📘 任務選單導航</strong></summary>

歡迎來到二進位購物資料的探索世界！  
以下是這份資料中可以進行的探索任務，歡迎選擇有興趣的項目進行分析：

1. 🏎️ 各商品分類的購買次數  
   — 統計各類商品的整體買氣，掌握熱銷類別的分布情形！

2. 🏆 購買次數最多的商品前10名  
   — 找出最常被選購的熱門商品，揭示人氣商品排行榜！

3. 🔁 回購分析  
   — 分析顧客對特定商品的重複購買行為，挖掘高黏著度商品與潛在忠誠偏好！

4. 👤 哪位顧客最常買哪個商品  
   — 探索顧客與商品的關聯，了解誰對哪些商品特別鍾情！

5. 🕵️ 每次買物最常在第幾步買了什麼  
   — 分析商品在購物流程中的出現位置，洞察決策時機與購買節奏！

6. 🔗 前後商品搭配出現頻率  
   — 檢視商品在購物順序中的搭配情形，找出常見的商品連動組合！

7. 🌊 Sankey Diagram：商品流動地圖  
   — 以 Sankey（桑基）圖視覺化商品在購物流程中的流動軌跡。

8. 📘 建立顧客故事小檔案  
   — 為每位顧客建立個人化的購物檔案，深入理解他們的消費風格與選擇邏輯。

</details>

## 1. 各商品分類的購買次數

統計各類商品的整體買氣，掌握熱銷品類趨勢！

In [ ]:
# 計算每個商品分類的購買次數
category_counts = df['category'].value_counts()
plt.figure(figsize=(8, 5))
ax = sns.barplot(y=category_counts.index, x=category_counts.values, hue=category_counts.index, palette="Set2", legend=False)

# 為每個 bar 加上邊框與數值標籤
for i, bar in enumerate(ax.patches):
    bar.set_edgecolor("black")
    bar.set_linewidth(0.8)
    width = bar.get_width()
    ax.text(width - 3, bar.get_y() + bar.get_height() / 2,
            f'{int(width)}', va='center', ha='right', fontsize=9, color='black', fontname='Verdana', weight='bold')

plt.xlabel("購買次數")
plt.ylabel("商品分類")
plt.title("各商品分類的購買次數", fontsize=14, weight='bold')
plt.grid(axis='x', linestyle='--', alpha=0.3)
plt.tight_layout()
# plt.savefig("../charts/category_counts.png")
plt.show()

## 2. 購買次數最多的商品（前 10 名）

瞭解最常被購買的熱門商品，找出人氣王！

In [ ]:
# 取得前 10 名購買次數最多的商品
product_counts = df['product'].value_counts().head(10)
plt.figure(figsize=(9, 5))
ax = sns.barplot(y=product_counts.index, x=product_counts.values, hue=product_counts.index, palette="Set3", legend=False)

# 為每個 bar 加上邊框與數值標籤
for i, bar in enumerate(ax.patches):
    bar.set_edgecolor("black")
    bar.set_linewidth(0.8)
    width = bar.get_width()
    ax.text(width - 3, bar.get_y() + bar.get_height() / 2,
            f'{int(width)}', va='center', ha='right', fontsize=9, color='black', fontname='Verdana', weight='bold')

plt.xlabel("購買次數")
plt.ylabel("商品名稱")
plt.title("購買次數最多的商品（前 10 名）", fontsize=14, weight='bold')
plt.grid(axis='x', linestyle='--', alpha=0.3)
plt.tight_layout()
# plt.savefig("../charts/top10_products.png")
plt.show()

## 3. 回購分析

找出顧客重複購買的商品，掌握高黏著度商品與偏好傾向！

In [ ]:
# 分析顧客對同一商品的回購次數
repurchase = (
    df.groupby(['customer_name', 'product'])
    .size()
    .reset_index(name='total_count')
)

repurchase = repurchase[repurchase['total_count'] > 1].copy()
repurchase['repurchase_count'] = repurchase['total_count'] - 1

repurchase = repurchase.sort_values(by='repurchase_count', ascending=False)
repurchase.head(10).reset_index(drop=True)

## 4. 哪位顧客最常買哪個商品

探索顧客的購買偏好，了解誰對哪個商品最熱愛！

In [ ]:
# 找出每位顧客購買次數最多的商品
customer_top_products = df.groupby(['customer_name', 'product']).size().reset_index(name='count')  # 顧客與商品的購買次數統計
customer_top1 = customer_top_products.sort_values(['customer_name', 'count'], ascending=[True, False])  # 對每位顧客依購買次數排序
customer_top1 = customer_top1.groupby('customer_name').head(1).reset_index(drop=True)  # 每位顧客取出購買次數最多的一項
customer_top1 = customer_top1.sort_values(by='count', ascending=False).reset_index(drop=True)  # 對結果依照購買次數再次排序
customer_top1.head(10)  # 顯示前 10 筆結果

## 5. 每次購物最常在第幾步買了什麼

觀察購物流程中最常出現的關鍵商品與時機點！

In [ ]:
# 找出每個購物步驟中出現次數最多的所有商品（即使有同分）
step_counts = (
    df.groupby(['sequence_order', 'product'])  # 依照購物步驟（sequence_order）與商品名稱（product）分組
    .size()  # 計算每組的出現次數，即在該步驟中該商品被購買的次數
    .reset_index(name='count')  # 將計算結果轉為 DataFrame，並命名為 'count'
)

# 對於每一個購物步驟，找出該步驟中最高的出現次數
max_per_step = step_counts.groupby('sequence_order')['count'].transform('max')

# 篩選出那些等於最高次數的商品（可能有多個）
top_by_step = step_counts[step_counts['count'] == max_per_step].reset_index(drop=True)

# 顯示每個購物步驟中最常被購買的商品
top_by_step

In [ ]:
# 列出所有在第１步驟出現的商品及其出現次數
df[df['sequence_order'] == 1]['product'].value_counts()

# 視覺化：每個購物步驟中最常購買的商品

In [ ]:
# 視覺化購物步驟與熱門商品對應關係
plt.figure(figsize=(9, 5))
ax = sns.barplot(data=top_by_step, x='sequence_order', y='count', hue='product', dodge=False, palette='Set1')

for bar in ax.patches:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width() / 2, height - 1,
            f'{int(height)}', ha='center', va='top', fontsize=9,
            color='white', fontname='Verdana', weight='bold')

plt.xlabel("購物步驟（sequence_order）")
plt.ylabel("購買次數")
plt.title("每次購物最常在第幾步買了什麼", fontsize=14, weight='bold')
plt.grid(axis='y', linestyle='--', alpha=0.3)
plt.tight_layout()
# plt.savefig("../charts/most_bought_by_step.png")
plt.show()

In [ ]:
# 額外交叉分析：在第三步購買牛奶餅乾的不同顧客數量
buyers_step3_snack = df[(df['sequence_order'] == 3) & (df['product'] == '牛奶餅乾')]['customer_name'].nunique()
print("在第 3 步購買牛奶餅乾的不同顧客數量：", buyers_step3_snack)

## 6. 前後商品搭配出現頻率

分析購物行為中商品的前後搭配模式，找出高頻出現的商品組合！

In [ ]:
# 🔗 前後商品搭配出現頻率（長條圖視覺化）

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)  # 忽略 FutureWarning 警告

# 過濾掉前後為相同商品的紀錄（避免自我搭配）
filtered_df = df[df['product'] != df['next_product']]

# 計算前後商品搭配的出現次數
pair_counts = (
    filtered_df.groupby(['product', 'next_product'])
    .size()
    .reset_index(name='count')
)

# 取出前 10 組最常見的商品搭配
top_pairs = pair_counts.sort_values(by='count', ascending=False).head(10)

top_pairs['pair'] = top_pairs['product'] + ' → ' + top_pairs['next_product']  # 建立易讀的商品組合標籤欄位

# 繪製長條圖
plt.figure(figsize=(10, 6))
ax = sns.barplot(x='count', y='pair', data=top_pairs, palette='viridis')

# 顯示每個長條圖右側的數字
for bar in ax.patches:
    width = bar.get_width()
    y = bar.get_y() + bar.get_height() / 2
    ax.text(width + 0.1, y, f'{int(width)}',  # 將數值標籤稍微向右側移動，避免與長條重疊
            va='center', ha='left', fontsize=10, fontweight='bold', color='#FF2800', fontname='Verdana')

# 設定標題與座標軸標籤
plt.xlabel("搭配出現次數")
plt.ylabel("商品組合")
plt.title("最常見的前後商品搭配組合（前10名）")

# 儲存圖檔
plt.tight_layout()
# plt.savefig("../charts/product_pairings.png")

# 顯示圖表
plt.show()

## 7. Sankey Diagram：商品流動地圖

以 Sankey（桑基）圖視覺化商品在不同購物步驟間的流動情形。

In [ ]:
import plotly.graph_objects as go

# 取出商品流動對（來源商品與下一商品）
sorted_df = df.sort_values(['session_id', 'sequence_order'])
sorted_df['next_product'] = sorted_df.groupby('session_id')['product'].shift(-1)
flow = sorted_df.dropna(subset=['next_product'])
flow = flow[flow['product'] != flow['next_product']]  # 🔍 排除自我搭配（如保溫瓶 ➝ 保溫瓶）

# 統計流動次數
flow_pairs = flow.groupby(['product', 'next_product']).size().reset_index(name='count')
top_flows = flow_pairs.sort_values(by='count', ascending=False).head(20)

# 建立節點
all_products = list(pd.unique(top_flows[['product', 'next_product']].values.ravel()))
product_indices = {product: i for i, product in enumerate(all_products)}

# 建立 Sankey 圖資料
sankey_data = go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=all_products
    ),
    link=dict(
        source=[product_indices[p] for p in top_flows['product']],
        target=[product_indices[p] for p in top_flows['next_product']],
        value=top_flows['count']
    )
)

# 繪製 Sankey 圖
fig = go.Figure(data=[sankey_data])
fig.update_layout(title_text="商品流動地圖（前 20 組）", font_size=10)
fig.show()


In [ ]:
# 進階：聚焦特定商品的流向（以牛奶餅乾為例）
target_product = "牛奶餅乾"
session_data = df.sort_values(['session_id', 'sequence_order'])

# 篩選出包含目標商品的購物場次
target_sessions = session_data[session_data['product'] == target_product]['session_id'].unique()
target_flow = session_data[session_data['session_id'].isin(target_sessions)]

# 產生前後配對關係
flow_pairs = target_flow.groupby('session_id')['product'].apply(lambda x: list(zip(x[:-1], x[1:]))).explode()
flow_counts = flow_pairs.value_counts().reset_index()
flow_counts.columns = ['pair', 'count']
flow_counts[['source', 'target']] = pd.DataFrame(flow_counts['pair'].tolist(), index=flow_counts.index)

# 準備 sankey 所需欄位
flow_summary = flow_counts[['source', 'target', 'count']]
flow_summary.head(10)

In [ ]:
import plotly.graph_objects as go

# 進階：聚焦特定商品的流向（以牛奶餅乾為例）
target_product = "牛奶餅乾"
session_data = df.sort_values(['session_id', 'sequence_order'])

# 篩選出包含目標商品的購物場次
target_sessions = session_data[session_data['product'] == target_product]['session_id'].unique()
target_flow = session_data[session_data['session_id'].isin(target_sessions)]

# 產生前後配對關係
flow_pairs = target_flow.groupby('session_id')['product'].apply(lambda x: list(zip(x[:-1], x[1:]))).explode()
flow_counts = flow_pairs.value_counts().reset_index()
flow_counts.columns = ['pair', 'count']
flow_counts[['source', 'target']] = pd.DataFrame(flow_counts['pair'].tolist(), index=flow_counts.index)

# 整理節點與編號
labels = pd.unique(flow_counts[['source', 'target']].values.ravel()).tolist()
label_index = {label: i for i, label in enumerate(labels)}

# 建立 Sankey 圖所需欄位
data = go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="lightblue"
    ),
    link=dict(
        source=[label_index[s] for s in flow_counts['source']],
        target=[label_index[t] for t in flow_counts['target']],
        value=flow_counts['count']
    )
)

# 畫圖
fig = go.Figure(data)
fig.update_layout(title_text="牛奶餅乾相關商品流向圖（Sankey Diagram）", font=dict(size=12))
fig.show()

## 8. 建立顧客故事小檔案

為每位顧客建立個人化的購物檔案，洞察他們的偏好與行為特徵。

In [ ]:
# 產出小美的購物紀錄摘要
df[df["customer_name"] == "小美"].groupby("product")["product"].count().sort_values(ascending=False)

In [ ]:
# 1. 小美的所有紀錄
xiaomei = df[df["customer_name"] == "小美"]

# 2. 購物次數（session_id 不重複）
shopping_sessions = xiaomei["session_id"].nunique()
print("購物次數：", shopping_sessions)

# 3. 總購買數量（總筆數）
total_items = xiaomei.shape[0]
print("總購買數量：", total_items)

# 4. 回購商品（只計算首次購買後再次出現的次數）
product_counts = xiaomei["product"].value_counts()
repurchase_items = (product_counts[product_counts > 1] - 1).astype(int)  # 扣除第一次購買
print("\n回購商品：")
print(repurchase_items)


### 👧 顧客故事小檔案：「小美」

📌 **基本軌跡**  
- 🛒 購物次數：274 次  
- 📦 總購買數量：280 件商品  
- 💡 回購商品：泡麵 (30 次)、果汁飲料 (25 次)、洗髮精 (25 次)、無糖紅茶 (21 次)、女用短襪 (20 次)、保溫瓶 (20 次)、護手霜 (19 次)、牛奶餅乾 (17 次)、牙刷組 (14 次)、男用短襪 (13 次)、洗手慕絲 (13 次)、洗衣精 (13 次)、螢光筆組 (12 次)、筆記本 (10 次)、無線滑鼠 (9 次)、快充行動電源 (8 次)、自動鉛筆 (7 次)、充電線 (4 次)

📊 **買物習性深探**  
- ⏱️ 最常在第 2 步做出購買決定

📝 **顧客行為敘述**  
小美是一位極具生活感的顧客，她的購物記錄像是一張日常生活的地圖。從三餐所需的泡麵與果汁飲料，到自我照護的洗髮精與護手霜，甚至還有辦公桌上的筆記本與螢光筆組，每一項商品都是她生活片段的縮影。

她常在購物的第二步就快速做出選擇，展現出她對日常所需的清晰認知與效率風格。雖然她回購最多的是泡麵與果汁飲料，但從其他商品的多樣性也可以看出她是個勇於嘗試、重視品質與便利性的消費者。

她的風格屬於「多元探索型」，在各種商品之間游刃有餘，既重視實用，也不失生活品味。若要為她設計推薦機制，搭配型商品組合與體驗式新品，可能最能吸引她的目光與點擊。

In [ ]:
def create_customer_summary(name):
    data = df[df['customer_name'] == name]  # 篩選指定顧客的資料
    total_sessions = data['session_id'].nunique()  # 計算購物場次
    total_items = data.shape[0]  # 計算總購買數量

    # 商品購買次數統計並排除首次購買
    product_counts = data['product'].value_counts()
    repurchases = (product_counts[product_counts > 1] - 1).astype(int)  # 回購次數 = 總次數 - 第一次購買

    top_step = (
        data['sequence_order']
        .value_counts()
        .sort_values(ascending=False)
        .index[0]  # 找出最常出現的購物步驟
    )

    print(f"📌 基本軌跡\n- 購物次數：{total_sessions} 次")
    print(f"- 總購買數量：{total_items} 件商品")
    if repurchases.empty:
        print("- 回購商品：無")
    else:
        repurchase_str = "、".join([f"{prod}（{cnt} 次）" for prod, cnt in repurchases.items()])
        print(f"- 回購商品：{repurchase_str}")

    print(f"\n📊 買物習性深探\n- 最常出現在第 {top_step} 步買商品")
    print(f"\n📝 顧客行為描述\n{name} 是個個性鮮明的顧客，接下來可以進一步探索她的購物邏輯與偏好商品喔！")

# 示範使用
create_customer_summary("小美")

In [ ]:
# 建立顧客故事小檔案的分析函式

def create_customer_summary(name):
    data = df[df['customer_name'] == name]  # 篩選指定顧客的資料
    total_sessions = data['session_id'].nunique()  # 計算購物場次
    total_items = data.shape[0]  # 計算總購買數量

    # 商品購買次數統計並排除第一次
    top_products = data['product'].value_counts()
    repurchases = (top_products[top_products > 1] - 1).astype(int)  # 回購次數 = 總次數 - 1

    top_step = (
        data['sequence_order']
        .value_counts()
        .sort_values(ascending=False)
        .index[0]  # 找出最常出現的購物步驟
    )

    print(f"📌 基本軌跡\n- 購物次數：{total_sessions} 次")
    print(f"- 總購買數量：{total_items} 件商品")
    if repurchases.empty:
        print("- 回購商品：無")
    else:
        repurchase_str = "、".join([f"{prod}（{cnt} 次）" for prod, cnt in repurchases.items()])
        print(f"- 回購商品：{repurchase_str}")

    print(f"\n📊 買物習性深探\n- 最常出現在第 {top_step} 步買商品")

    # 自動歸類敘述
    categories = data['category'].value_counts()
    if not categories.empty:
        top_cat = categories.idxmax()
        if top_cat in ['保健', '日常用品', '食品']:
            style = "日常實用型"
        elif top_cat in ['3C產品', '數位周邊']:
            style = "科技導向型"
        elif top_cat in ['美容保養', '個人清潔']:
            style = "自我照護型"
        else:
            style = "多元探索型"
        print(f"\n📝 顧客行為描述\n{name} 展現出「{style}」的購物風格，值得深入了解其偏好與決策模式！")
    else:
        print(f"\n📝 顧客行為描述\n{name} 是個個性鮮明的顧客，接下來可以進一步探索她的購物邏輯與偏好商品喔！")

# 示範使用
create_customer_summary("小美")

## 🔍 資料簡要總覽

In [ ]:
print("資料筆數與欄位數：", df.shape)  # 顯示資料的列數與欄位數
print(df.info())  # 顯示每個欄位的型別與非空值數量

# 顯示每位顧客的購物場次數量（用 session_id 去重計算）
print("\n各顧客購物場次統計：")
print(df.groupby("customer_name")["session_id"].nunique())

# 顯示每位顧客購買商品的總次數（不管是哪次購物）
print("\n各顧客總購買數量：")
print(df["customer_name"].value_counts())